In [ ]:
!pip install evaluate rouge-score nltk
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U transformers==4.38.0

In [ ]:
pip install -q -U datasets==2.16.1

In [ ]:
import transformers

print(transformers.__version__)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
from transformers.utils import send_example_telemetry

send_example_telemetry("summarization_notebook", framework="pytorch")

In [2]:
model_checkpoint = "facebook/bart-large-cnn"

In [3]:
from datasets import load_dataset
from evaluate import load

raw_datasets = load_dataset("ccdv/pubmed-summarization")
metric = load("rouge")

2024-03-19 10:33:00.454367: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-19 10:33:00.454431: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-19 10:33:00.455858: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for ccdv/pubmed-summarization contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ccdv/pubmed-summarization
You can avoid this message in future by passing the argument `trust_remote_c

In [4]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract'],
        num_rows: 119924
    })
    validation: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6633
    })
    test: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6658
    })
})

In [5]:
raw_datasets["train"][0]

{'article': "a recent systematic analysis showed that in 2011 , 314 ( 296 - 331 ) million children younger than 5 years were mildly , moderately or severely stunted and 258 ( 240 - 274 ) million were mildly , moderately or severely underweight in the developing countries . \n in iran a study among 752 high school girls in sistan and baluchestan showed prevalence of 16.2% , 8.6% and 1.5% , for underweight , overweight and obesity , respectively . \n the prevalence of malnutrition among elementary school aged children in tehran varied from 6% to 16% . \n anthropometric study of elementary school students in shiraz revealed that 16% of them suffer from malnutrition and low body weight . \n snack should have 300 - 400 kcal energy and could provide 5 - 10 g of protein / day . nowadays , school nutrition programs are running as the national programs , world - wide . national school lunch program in the united states \n there are also some reports regarding school feeding programs in developi

In [6]:
metric

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [7]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [9]:
tokenizer("Hello, this one sentence!")

{'input_ids': [0, 31414, 6, 42, 65, 3645, 328, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [10]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[0, 31414, 6, 42, 65, 3645, 328, 2], [0, 713, 16, 277, 3645, 4, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}

In [11]:
if model_checkpoint in ["t5-small", "t5-base", "google-t5/t5-small", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [12]:
prefix

''

In [13]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["abstract"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [14]:
preprocess_function(raw_datasets['train'][:1])

{'input_ids': [[0, 102, 485, 20552, 1966, 969, 14, 11, 1466, 2156, 36835, 36, 40625, 111, 39507, 4839, 153, 408, 3240, 87, 195, 107, 58, 33019, 2156, 30389, 50, 11166, 16990, 5357, 8, 36586, 36, 15452, 111, 38415, 4839, 153, 58, 33019, 2156, 30389, 50, 11166, 223, 4301, 11, 5, 2623, 749, 479, 1437, 50118, 11, 10209, 260, 10, 892, 566, 262, 4429, 239, 334, 1972, 11, 579, 7566, 8, 17832, 3964, 20921, 260, 969, 21087, 9, 545, 4, 176, 207, 2156, 290, 4, 401, 207, 8, 112, 4, 245, 207, 2156, 13, 223, 4301, 2156, 19755, 8, 14057, 2156, 4067, 479, 1437, 50118, 5, 21087, 9, 29638, 566, 10891, 334, 5180, 408, 11, 3055, 43966, 14903, 31, 231, 207, 7, 545, 207, 479, 1437, 50118, 32117, 22356, 892, 9, 10891, 334, 521, 11, 1481, 853, 1222, 1487, 14, 545, 207, 9, 106, 6297, 31, 29638, 8, 614, 809, 2408, 479, 1437, 50118, 17687, 197, 33, 2993, 111, 3675, 48130, 1007, 8, 115, 694, 195, 111, 158, 821, 9, 8276, 1589, 183, 479, 25708, 2156, 334, 10894, 1767, 32, 878, 25, 5, 632, 1767, 2156, 232, 111, 1810

In [15]:
tokenizer.decode([0, 46776, 4832, 5, 1455, 892, 21, 2584, 66, 7, 7118, 5, 3038, 9, 435, 10894, 6530, 716, 15, 8973, 1548, 15, 29638, 2194, 566, 334, 111, 5180, 408, 11, 1481, 853, 1222, 2156, 10209, 260, 4, 36739, 29, 8, 6448, 4832, 42, 403, 111, 797, 22813, 6530, 34, 57, 626, 227, 2266, 8, 2338, 15, 971, 6750, 2270, 8, 5929, 334, 2786, 8, 1972, 36, 262, 111, 508, 107, 793, 4839, 716, 15, 8973, 1548, 11, 1481, 853, 1222, 2156, 10209, 260, 479, 1437, 50118, 5, 695, 1286, 30426, 14967, 11, 285, 1304, 81, 10, 132, 12, 180, 675, 552, 19, 8973, 28094, 2163, 11, 645, 7, 5731, 8, 3720, 22813, 6530, 479, 13, 10437, 9, 12833, 9, 5, 6530, 434, 4872, 21383, 9, 1198, 12, 2])

'<s>background : the present study was carried out to assess the effects of community nutrition intervention based on advocacy approach on malnutrition status among school - aged children in shiraz, iran.materials and methods : this case - control nutritional intervention has been done between 2008 and 2009 on 2897 primary and secondary school boys and girls ( 7 - 13 years old ) based on advocacy approach in shiraz, iran. \n the project provided nutritious snacks in public schools over a 2-year period along with advocacy oriented actions in order to implement and promote nutritional intervention. for evaluation of effectiveness of the intervention growth monitoring indices of pre-</s>'

In [16]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

In [17]:
import pickle

# with open('/kaggle/working/tokenized_datasets_inp.pickle', 'wb') as f:
#     pickle.dump(tokenized_datasets, f)

In [18]:
# Load data from the pickle file
with open('/kaggle/working/tokenized_datasets_inp.pickle','rb') as f:
     tokenized_datasets = pickle.load(f)

print("Data loaded from data.pickle:", tokenized_datasets)

Data loaded from data.pickle: DatasetDict({
    train: Dataset({
        features: ['article', 'abstract', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 119924
    })
    validation: Dataset({
        features: ['article', 'abstract', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 6633
    })
    test: Dataset({
        features: ['article', 'abstract', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 6658
    })
})


In [19]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [20]:
# Define hyperparameters
batch_size = 2
model_name = model_checkpoint.split("/")[-1]
learning_rate = 5e-5
weight_decay = 0.01
num_train_epochs = 1
logging_steps = 5
gradient_accumulation_steps = 16

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-xsum",
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=weight_decay,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    gradient_accumulation_steps=gradient_accumulation_steps,
    fp16=True,  # Enable mixed precision training if supported
    push_to_hub=False,
)

In [21]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [22]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [23]:
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

wandb: Currently logged in as: vickyvijay069 (vicky12). Use `wandb login --relogin` to force relogin


In [ ]:
trainer.model.save_pretrained("/kaggle/working/summarizationV2-model")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer1 = AutoTokenizer.from_pretrained("suriya7/bart-finetuned-text-summarization")
model1 = AutoModelForSeq2SeqLM.from_pretrained("suriya7/bart-finetuned-text-summarization")

def generate_summary(text):
    inputs = tokenizer1([text], max_length=1024, return_tensors='pt', truncation=True)
    summary_ids = model1.generate(inputs['input_ids'], max_new_tokens=100, do_sample=False)
    summary = tokenizer1.decode(summary_ids[0], skip_special_tokens=True)
    return summary

text_to_summarize = raw_datasets["test"]['article'][0]
summary = generate_summary(text_to_summarize)
print(summary)

In [ ]:
raw_datasets["test"]['abstract'][0]

In [ ]:
model2 = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/working/summarizationV2-model")

In [ ]:
def generate_summary(text):
    inputs = tokenizer([text], max_length=1024, return_tensors='pt', truncation=True)
    summary_ids = model2.generate(inputs['input_ids'], max_new_tokens=100, do_sample=False)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [ ]:
text_to_summarize = raw_datasets["test"]['article'][10]
summary = generate_summary(text_to_summarize)
print(summary)

In [ ]:
raw_datasets["test"]['abstract'][10]